# Introduction

[Instrumentation ticket](https://phabricator.wikimedia.org/T294777)  |  [QA ticket](https://phabricator.wikimedia.org/T297921)

# Instrumentation note
Web team has deployed the instrumentation to measure read depth of anonymous user on mobile web talk page. 
The related events will be stored in `event.mediawiki_reading_depth` schema. Sample rate is 0.1% on English Wikipedia. 



# Table of Contents



In [1]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
library(tidyverse); 
library(lubridate); 
library(scales);
library(magrittr); 
library(dplyr);
})

In [103]:
library(IRdisplay)

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
 </form>'
)

In [99]:
options(repr.plot.width = 15, repr.plot.height = 10)

__QA on 12/16/2021__

# Check daily events

In [3]:
query <- 
"
SELECT TO_DATE(dt),  year, month,day, COUNT(1) AS events, 
COUNT(DISTINCT session_token) AS sessions
FROM event.mediawiki_reading_depth
WHERE year=2021
GROUP BY TO_DATE(dt),year, month,day
"

In [4]:
df <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [5]:
df

X_c0,year,month,day,events,sessions
<chr>,<int>,<int>,<int>,<int>,<int>
2021-11-02,2021,12,3,2,1
2021-11-11,2021,12,2,2,1
2021-11-11,2021,12,10,2,1
2021-11-17,2021,11,23,2,1
2021-11-17,2021,11,27,1,1
2021-11-17,2021,12,4,4,1
2021-11-18,2021,12,3,17,1
2021-11-19,2021,11,25,2,1
2021-11-19,2021,11,29,4,2


__Note:__  

The data in dt field doesn't match with the partition year, month day. It stored the date as early as 1986. It also stored future dates, in 2022, etc.



QAed on 12-20-2021

__Reason:__ 

In modern eventloggin platform dt has switched to meaning the time according to the client. So if someone, say, has the time on their phone set to 2008 or 2022, dt would reflect that. On the other hand, meta.dt (which is used to set the partition fields) is the time our server received the event, which would still be 2021. [ticket](https://phabricator.wikimedia.org/T292586#7581979)

Work-around: query data using partitions or meta.dt instead of dt field

In [14]:
query <- 
"
SELECT to_date(meta.dt) AS date_time, year, month,day, COUNT(1) AS events, 
COUNT(DISTINCT session_token) AS sessions
FROM event.mediawiki_reading_depth
WHERE year=2021
GROUP BY to_date(meta.dt), year, month,day
ORDER BY year, month,day
LIMIT 100000
"

In [15]:
df <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [16]:
df

date_time,year,month,day,events,sessions
<chr>,<int>,<int>,<int>,<int>,<int>
2021-11-23,2021,11,23,67851,16819
2021-11-24,2021,11,24,253034,61902
2021-11-25,2021,11,25,256473,64688
2021-11-26,2021,11,26,252048,64375
2021-11-27,2021,11,27,283098,70795
2021-11-28,2021,11,28,305501,74499
2021-11-29,2021,11,29,301952,74162
2021-11-30,2021,11,30,309143,75005
2021-12-01,2021,12,1,305327,72637


__Note:__  
Events are avaiable sicne 2021-11-23

# By wiki

In [17]:
query <- 
"
SELECT meta.domain, COUNT(1) AS events, 
COUNT(DISTINCT session_token) AS sessions
FROM event.mediawiki_reading_depth
WHERE year=2021
GROUP BY meta.domain
ORDER BY meta.domain
LIMIT 100000
"

In [19]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [20]:
df

domain,events,sessions
<chr>,<int>,<int>
en.jinzhao.wiki,245,18
en.turkcewiki.org,4,2
en.wiki.hancel.org,1197,8
en.wikidark.org,2,1
en.wikipedia.ahau.cf,10,2
en.wikipedia.ahmu.cf,2934,1
en.wikipedia.ahut.cf,8,1
en.wikipedia.hfut.cf,6,2
en.wikipedia.iwiki.eu.org,14,6


__Note:__  
Only enabled on English wikipedia

# By namespace

In [21]:

query <- 
"
SELECT page_namespace, COUNT(1) AS events, 
COUNT(DISTINCT session_token) AS sessions
FROM event.mediawiki_reading_depth
WHERE year=2021
GROUP BY page_namespace
ORDER BY page_namespace
LIMIT 100000
"


In [22]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [23]:
df

page_namespace,events,sessions
<int>,<int>,<int>
-1,109583,22327
0,10910210,2499386
1,15535,6217
2,7708,1352
3,3959,449
4,19451,5376
5,487,115
6,30640,11024
7,121,61


# BY user type: anonymous users and logged-in users

In [26]:
query <- 
"
SELECT is_anon, COUNT(1) AS events, 
COUNT(DISTINCT session_token) AS sessions
FROM event.mediawiki_reading_depth
WHERE year=2021
GROUP BY is_anon
LIMIT 100000
"


In [27]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [28]:
df


is_anon,events,sessions
<chr>,<int>,<int>
false,150123,6825
true,11090179,2505231


# By action

In [29]:

query <- 
"
SELECT action, COUNT(1) AS events, 
COUNT(DISTINCT session_token) AS sessions
FROM event.mediawiki_reading_depth
WHERE year=2021
GROUP BY action
LIMIT 100000
"


In [30]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [31]:
df

action,events,sessions
<chr>,<int>,<int>
pageLoaded,7230343,2507202
pageUnloaded,4009959,961392


__Note:__  

Two types of actions are recorded: pageLoaded, pageUnloaded

# By page_length

In [32]:
query <- 
"
SELECT page_length, COUNT(1) AS events, 
COUNT(DISTINCT session_token) AS sessions
FROM event.mediawiki_reading_depth
WHERE year=2021
GROUP BY page_length
ORDER BY page_length
LIMIT 100000
"


In [33]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [34]:
df

page_length,events,sessions
<int>,<int>,<int>
-1,10568,5367
0,149593,35760
1,20,6
2,12,2
4,19,2
5,13,5
6,8,3
7,2,1
8,16,1


# By access method

In [37]:
query <- 
"
SELECT access_method, COUNT(1) AS events, 
COUNT(DISTINCT session_token) AS sessions
FROM event.mediawiki_reading_depth
WHERE year=2021
GROUP BY access_method
LIMIT 100000
"


In [38]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [39]:
df

access_method,events,sessions
<chr>,<int>,<int>
desktop,4387438,570453
mobile web,6852864,1940983


__Note:__

Two types of access methods are recorded: desktop, mobile web

# By agent type

In [42]:
query <- "
SELECT CASE WHEN user_agent_map['device_family']='Spider' THEN 'Spider' ELSE 'User' END AS agent_type,
COUNT(1) AS events,
COUNT(DISTINCT session_token) AS sessions
FROM event.mediawiki_reading_depth
WHERE year=2021
GROUP BY CASE WHEN user_agent_map['device_family']='Spider' THEN 'Spider' ELSE 'User' END
LIMIT 100000
"



In [43]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [44]:
df

agent_type,events,sessions
<chr>,<int>,<int>
Spider,141,114
User,11240161,2511334


__Note:__

We can identify and exclude spider in analysis

# Read length Distribution

In [58]:
query_total_length <- 
"
SELECT MAX(total_length) AS max, MIN(total_length) AS min, Avg(total_length) AS avg, STDDEV(total_length) AS stdev, COUNT(1) AS count
FROM event.mediawiki_reading_depth
WHERE year=2021 AND total_length is not NULL
"


In [59]:
df <- wmfdata::query_hive(query_total_length)

Don't forget to authenticate with Kerberos using kinit



In [60]:
df

max,min,avg,stdev,count
<dbl>,<int>,<dbl>,<dbl>,<int>
2603264579,0,1178408,16303164,4009959


__Note:__

For all events in 2021, the average reading length is 1178408 MS

In [76]:
query_total_length <- 
"
SELECT total_length, session_token
FROM event.mediawiki_reading_depth
WHERE year=2021 
AND total_length is not NULL
"


In [77]:
df <- wmfdata::query_hive(query_total_length)

Don't forget to authenticate with Kerberos using kinit



In [118]:
# Represent it

p <- df %>%
  ggplot( mapping=aes(x=total_length)) +
    geom_histogram(bins=30 ) +
    scale_x_log10() +
    labs ( title = "Check distribution of total read length",
         fill="") +
    theme_light(base_size=18)


In [119]:
ggsave("tmp.png",
    plot = p, width = 60, height = 30, units = "cm", dpi = "screen");

Warning message:
“Transformation introduced infinite values in continuous x-axis”
Warning message:
“Removed 2 rows containing non-finite values (stat_bin).”
